In [6]:

from gui import GUI
import tkinter

# Don't execute this if file is imported
if __name__ == '__main__':
    gui = GUI()
    gui.show()
    # make_image(gui, color_mandel)
    tkinter.mainloop()
    


from typing import List, Tuple, Sequence
from numba import jit
import math


#: The type of 2D points.
Point = Tuple[float, float]


#: TODO: define this function
@jit(nopython=True)
def mandel_seq(x: float, y: float, n: int = 100) -> Sequence[Point]:
    """Return the mandel sequence for the input point (x, y), using n as upper bound.
    Assumptions:
    * start the sequence at (u_0, v_0) = (0, 0)
    * The coordinate (x, y) will have mandel number n,
      when the sequence starts diverging at (u_n,  v_n)
    :param x: x-coordinate of the point for which the sequence is computed
    :param y: y-coordinate of the point for which the sequence is computed
    :param n: upper bound to detect divergence
    :return: the mandel sequence for the point (x, y)
    :examples:
    >>> mandel_seq(1, 0)
    [(0.0, 0.0), (1.0, 0.0), (2.0, 0.0), (5.0, 0.0)]
    >>> mandel_seq(0, 0, n = 1)
    [(0.0, 0.0), (0.0, 0.0)]
    """

    (u_0, v_0) = (0.0, 0.0)
    result = [(u_0, v_0)]
    for i in range(n):
        v_n = x + result[i][0]**2 - result[i][1]**2
        u_n = y + 2*result[i][0]*result[i][1]
        result.append((v_n, u_n))
        if u_n**2 + v_n**2 > 4:
            break

    return result


#: TODO: define this function
@jit(nopython=True)
def mandel_number(x: float, y: float, n: int = 100) -> int:
    """Return the mandel-number of point (x, y).
    This is the smallest index of the mandel sequence at which u_n^2 + v_n^2 > 4.
    Assumptions:
    * the sequence diverges when u_n^2 + v_n^2 > 4
    :param x: x-coordinate of the point for which the Mandel number is computed
    :param y: y-coordinate of the point for which the Mandel number is computed
    :param n: upper bound to detect divergence
    :return: the mandel-number of point (x, y).
    :examples:
    >>> mandel_number(1, 0)
    3
    >>> mandel_number(0, 0, n = 10)
    10
    """
    return len(mandel_seq(x, y, n)) - 1


#: The type of colors
Color = Tuple[int, int, int]
#: The standard color
STDC: Color = (140, 0, 255)
#: Black
BLACK: Color = (0, 0, 0)
#: Grey
GREY: Color = (128, 128, 128)
#: White
WHITE: Color = (255, 255, 255)
#: Red
RED: Color = (255, 0, 0)
#: Green
GREEN: Color = (0, 100, 0)
#: Blue
BLUE: Color = (0, 0, 255)


#: TODO: define this function
#: TODO: document this
@jit(nopython=True)
def convert_pixel(x: int, y: int, cx: float = 0.0, cy: float = 0.0, zoom: float = 1) -> Point:
    """
    :param zoom: scale based on x range
    :param cy: center y-axis
    :param cx: center x-axis
    :param x: locate given X in x-axis
    :param y: locate given Y in Y-axis
    :return: location of pixel in xy-plane
    """
    result_x = 0.0
    result_y = 0.0
    if x <= 600 and y <= 600:
        result_x = (x - 300) * zoom / 600
        result_y = (y - 300) * zoom / -600
    return result_x + cx, result_y + cy


#: TODO: define this function
#: TODO: document this
def generate_mandel_nums(cx: float = 0.0, cy: float = 0.0, zoom: float = 1) -> List[List[int]]:
    """ Create nested list that represent a 600 x 600 pixel image
        :return: nested list
    """
    img_memory = [
        [mandel_number(convert_pixel(i, j, cx, cy, zoom)[0], convert_pixel(i, j, cx, cy, zoom)[1]) for i in range(600)]
        for j in range(600)]
    return img_memory


#: TODO: define this function
@jit(nopython=True)
def color_mandel(px: int, py: int, n: int = 100, c1: Color = GREEN, cx: float = 0, cy: float = 0,
                 zoom: float = 1) -> Color:
    """Return the colour of the given pixel.
    Assumptions:
    * pixels that that diverge immediately -> BLACK
    * pixels that do not diverge after n iterations -> BLACK
    * colour scaled based on the squared root of the mandel number
    :param: cp: center point
    :param: px: pixel x-coordinate
    :param: py: pixel y-coordinate
    :param: n: maximum number of iterations, after which the pixel is coloured BLACK
    :return: Color(as a defined Tuple - lines 48) for the input pixel
    :examples:
    TODO: doctests
    """
    h = mandel_number(convert_pixel(px, py, cx, cy, zoom)[0], convert_pixel(px, py, cx, cy, zoom)[1])
    sh = int(math.sqrt(h) * 20)
    r, g, b = c1[0] + sh, c1[1] + sh, c1[2] + sh
    if r > 255:
        r = 255
    if g > 255:
        g = 255
    if b > 255:
        b = 255
    if h == 100:
        c = BLACK
    elif h <= 1:
        c = BLACK
    else:
        c = (r, g, b)
    return c



# Author: Lars van den Haak and Tom Verhoeff
# Copyright (c) 2020 - Eindhoven University of Technology, The Netherlands
# This software is made available under the terms of the MIT License.
# * Contributor 1: M.J. Moshiri
# * TU/e ID number 1: 1524003
# * Contributor 2: M.G.A. Fatah
# * TU/e ID number 2: 1297546
# * Date: ...
# """
from PIL import Image, ImageTk, ImageGrab  # type: ignore
import tkinter as tk
from tkinter.ttk import Progressbar
from mandel import *
from typing import Callable

# Store information about visualized fraction as global
visualized_x_c = ""
visualized_y_c = ""
visualized_zoom = ""
visualized_range_a = ""
visualized_range_b = ""
image_cache = Image.new('RGB', (600, 600))

color_bank = [['Red', RED],
              ['Blue', BLUE],
              ['Green', GREEN],
              ['Grey', GREY],
              ['Black', BLACK],
              ['STDC', STDC]
              ]
choices = [color_bank[i][0] for i in range(len(color_bank))]


class GUI:
    """ A class where we make our Graphical User Interface based on TkInter
    """
    def __init__(self) -> None:
        self.image = None
        self.window = tk.Tk()
        self.canvas = tk.Label()
        self.canvas.grid(row=0, column=1)
        self.canvas.bind('<Button-1>', self.zoom_action)
        self.canvas.bind('<Button-3>', self.zoom_action)
        self.window.title('Mandelbrot visualiser')
        self.window.resizable(False, False)
        self.window.grid_columnconfigure(1, minsize=600)
        self.window.grid_rowconfigure(0, minsize=600)

        # Panel Interface
        self.main_panel = tk.LabelFrame(self.window, height=600, width=100, text="Panel",
                                        font=("Helvetica", 10, "bold italic"))
        self.main_panel.grid(row=0, column=0, sticky='nsw', padx=5)

        # Drop-down Interface
        self.chosen_color = tk.StringVar()
        self.chosen_color.set(choices[2])
        self.color_option = tk.OptionMenu(self.main_panel, self.chosen_color, *choices)
        self.label_color = tk.Label(self.main_panel, text="Color:")
        self.label_color.grid(row=0, column=0)
        self.color_option.grid(row=0, column=1, sticky='nsew', padx=5)

        # Center X and Y interface
        self.xy_panel = tk.Frame(self.main_panel)
        self.xy_panel.grid(row=1, column=1, padx=10, pady=5)
        self.center_x = tk.StringVar()
        self.center_y = tk.StringVar()
        self.label_center = tk.Label(self.main_panel, text="Center:")
        self.label_center.grid(row=1, column=0, sticky='nsew')
        self.label_x_center = tk.Label(self.xy_panel, text="X:")
        self.label_x_center.grid(row=0, column=0, sticky='w')
        self.c_x = tk.Entry(self.xy_panel, textvariable=self.center_x, justify='center')
        self.c_x.grid(row=0, column=1, sticky='nsew')
        self.label_y_center = tk.Label(self.xy_panel, text="Y:")
        self.label_y_center.grid(row=1, column=0, sticky='w')
        self.c_y = tk.Entry(self.xy_panel, textvariable=self.center_y, justify='center')
        self.c_y.grid(row=1, column=1, padx=1, pady=5, sticky='e')
        self.c_x.insert(0, "-0.5")
        self.c_y.insert(0, "0.0")

        # Define Zoom/Range switch
        self.active_form = tk.Button(self.main_panel, text="Scale:", relief='sunken', command=self.chang_form)
        self.active_form.grid(row=2, column=0, sticky="SEW", padx=5)

        # Define ZoomPanel Interface
        self.zoom_panel = tk.Frame(self.main_panel)
        self.zoom_panel.grid(row=2, column=1, padx=5)
        self.scale = tk.StringVar()
        self.zoom = tk.Entry(self.zoom_panel, textvariable=self.scale, width=10, justify='center')
        self.zoom.grid(row=2, column=1, padx=5)
        self.zoom.insert(0, 2)

        # Define RangePanel Interface
        self.range_panel = tk.Frame(self.main_panel)
        self.range_a = tk.StringVar()
        self.range_b = tk.StringVar()
        self.range_f_a = tk.Entry(self.range_panel, textvariable=self.range_a, width=7, justify='center')
        self.range_f_a.grid(row=0, column=0, sticky='w')
        self.label_to = tk.Label(self.range_panel, text="TO")
        self.label_to.grid(row=0, column=1, padx=5)
        self.range_f_b = tk.Entry(self.range_panel, textvariable=self.range_b, width=7, justify='center')
        self.range_f_b.grid(row=0, column=2, sticky='e')

        # Visualise btn Interface
        self.btn_vis = tk.Button(self.main_panel, text="Visualise", command=self.show)
        self.btn_vis.grid(row=4, column=1, sticky='sew', padx=5, pady=5)
        self.btn_blur = tk.Button(self.main_panel, text="blur", bg="lightblue", command=self.blur)
        self.btn_blur.grid(row=4, column=0, sticky='snwe', padx=5, pady=5)

        # Saving Interface
        self.label_export = tk.Label(self.main_panel, text="Output name:")
        self.label_export.grid(row=5, column=0)
        self.name_in = tk.StringVar()
        self.file_name = tk.Entry(self.main_panel, textvariable=self.name_in)
        self.file_name.grid(row=5, column=1, sticky='sew', padx=5, pady=5)
        self.btn_save = tk.Button(self.main_panel, text="Save", command=self.save_image)
        self.btn_save.grid(row=9, columnspan=2, sticky='sew', padx=5, pady=5)

        # Reset Interface
        self.btn_reset = tk.Button(self.main_panel, text="Reset", command=self.reset)
        self.btn_reset.grid(row=10, columnspan=2, sticky='sew', padx=5, pady=5)


        # Progress
        self.progress = Progressbar(self.window, orient=tk.HORIZONTAL, length=600, mode='determinate', maximum=600)
        self.progress.grid(row=3, column=0, sticky='sew', columnspan=2)

    def reset(self):
        self.c_x.delete(0, tk.END)
        self.c_y.delete(0, tk.END)
        self.zoom.delete(0, tk.END)

        self.c_x.insert(0, "-0.5")
        self.c_y.insert(0, "0.0")
        self.zoom.insert(0, 2)
        self.show()

    def chang_form(self):
        """ Change the visibilty of inputboxes to see what user wants (scale/range)
        """
        if self.active_form['text'] == "Scale:":
            self.active_form.config(relief="raised", text="Range:")
            self.zoom_panel.grid_forget()
            self.range_panel.grid(row=2, column=1, padx=5)
        else:
            self.active_form.config(relief="sunken", text="Scale:")
            self.range_panel.grid_forget()
            self.zoom_panel.grid(row=2, column=1, padx=5)

    def save_image(self) -> None:
        """ Read pixels in canvas and save is as PNG pic
        """
        filename = self.name_in.get()
        if filename == "":
            filename = "Default"
        # x = self.window.winfo_rootx() + self.canvas.winfo_x()
        # y = self.window.winfo_rooty() + self.canvas.winfo_y()
        # xx = x + self.canvas.winfo_width()
        # yy = y + self.canvas.winfo_height()
        # ImageGrab.grab(bbox=(x, y, xx, yy)).save(+ ".PNG")
        image_cache.save(filename + ".jpg")

    def show(self) -> None:
        """ Revisualise the manderlbrot with chosen color and coordinates
        """
        global visualized_x_c, visualized_y_c, visualized_zoom, visualized_range_a, visualized_range_b
        color = self.chosen_color.get()
        # User has the option to choose between scale and range.
        if self.active_form['text'] == "Scale:":
            visualized_zoom = abs(float(self.scale.get()))
            visualized_x_c = float(self.center_x.get())
            visualized_y_c = float(self.center_y.get())
        else:
            visualized_zoom = abs(float(self.range_f_b.get()) - float(self.range_f_a.get()))
            visualized_x_c = float(self.range_f_a.get()) + visualized_zoom / 2
            visualized_y_c = float(self.center_y.get())
            self.c_x.delete(0, tk.END)
            self.c_x.insert(0, visualized_x_c)
            self.zoom.delete(0, tk.END)
            self.zoom.insert(0, visualized_zoom)

        vis_col = (color_bank[0][1])  # Default value for variable to not to be empty
        # Find appropriate color tuple based on the string given
        for i in range(len(color_bank)):
            if color_bank[i][0] == color:
                vis_col = (color_bank[i][1])
        global image_cache
        img = Image.new('RGB', (600, 600))
        i = 0
        # global img_cache
        # if len(img_cache) == 0:
        #     img_cache = generate_mandel_nums(cx=float(center_x), cy=float(center_y), zoom=distance)

        for x in range(0, 600):
            i += 1
            self.progress["value"] = int(i)
            self.progress.update()
            self.btn_vis['text'] = "Visualising: " + str(int(i / 6)) + "%"
            for y in range(0, 600):
                # img.putpixel((x, y), paint(img_cache[y][x], vis_col))
                image_cache.putpixel((x, y), color_mandel(x, y,
                                                          c1=vis_col,
                                                          cx=float(visualized_x_c),
                                                          cy=float(visualized_y_c),
                                                          zoom=visualized_zoom))

        tkimg = ImageTk.PhotoImage(image_cache)
        self.image = tkimg
        canvas = self.canvas
        canvas.configure(image=tkimg)
        # canvas.grid(row=0, column=1)
        self.btn_vis['text'] = "Visualise"
        self.label_center['text'] = "Center :"
        self.label_center.config(foreground="Black")
        self.c_x.config(foreground="Black")
        self.c_y.config(foreground="Black")
        self.range_coordinate()

    def zoom_action(self, event):
        """ Add mouse position as coordinates in X-Y and add it to text box with red Color
            Check the event to see if its a zoom in or zoom out call
            :param event: mouse click
        """
        if event.num == 1:
            scale_number = 1 / 2
        elif event.num == 3:
            scale_number = 2
        global visualized_x_c, visualized_y_c, visualized_zoom
        distance = abs(float(self.zoom.get()))
        new_coordinates = convert_pixel(event.x, event.y, zoom=distance)
        self.c_x.delete(0, tk.END)
        self.c_y.delete(0, tk.END)
        self.label_center['text'] = "New Center :"
        self.label_center.config(foreground="red")
        self.c_x.config(foreground="red")
        self.c_y.config(foreground="red")
        self.c_x.insert(0, new_coordinates[0] + visualized_x_c)
        self.c_y.insert(0, new_coordinates[1] + visualized_y_c)
        visualized_zoom = abs(float(self.zoom.get())) * scale_number
        self.zoom.delete(0, tk.END)
        self.zoom.insert(0, visualized_zoom)
        self.show()

    def range_coordinate(self):
        """ Find the visualized fill the range value based on center point and scale
        """
        global visualized_x_c, visualized_zoom, visualized_range_a, visualized_range_b
        visualized_range_a = visualized_x_c - visualized_zoom / 2
        visualized_range_b = visualized_x_c + visualized_zoom / 2
        self.range_f_a.delete(0, tk.END)
        self.range_f_b.delete(0, tk.END)
        self.range_f_a.insert(0, visualized_range_a)
        self.range_f_b.insert(0, visualized_range_b)

    def blur(self):
        global image_cache
        blur_image = image_cache.load()
        pixels = list(image_cache.getdata())
        width, height = image_cache.size
        pixels = [pixels[i * width:(i + 1) * width] for i in range(height)]
        # print(pixels)
        img = Image.new('RGB', (600, 600))
        for x in range(0, 599):
            for y in range(0, 599):
                color = []
                color.insert(0, int((list(blur_image[x - 1, y])[0] +
                                     list(blur_image[x, y])[0] +
                                     list(blur_image[x + 1, y])[0]) / 3))
                color.insert(1, int((list(blur_image[x - 1, y])[1] +
                                     list(blur_image[x, y])[1] +
                                     list(blur_image[x + 1, y])[1]) / 3))
                color.insert(2, int((list(blur_image[x - 1, y])[2] +
                                     list(blur_image[x, y])[2] +
                                     list(blur_image[x + 1, y])[2]) / 3))
                img.putpixel((x, y), tuple(color))

        tkimg = ImageTk.PhotoImage(img)
        image_cache = img
        self.image = tkimg
        canvas = self.canvas
        canvas.configure(image=tkimg)

# def make_image(gui: GUI, colorize: Callable[[int, int], Color] = color_mandel) -> None:
#     """Puts an image on screen created by a function
#
#     :param: gui: An instance from the GUI class
#     :param: colorize: A function that gives a color to each pixel
#     """
#     img = Image.new('RGB', (600, 600))
#     for x in range(0, 600):
#         for y in range(0, 600):
#             img.putpixel((x, y), colorize(x, y))
#
#     tkimg = ImageTk.PhotoImage(img)
#     # Save the image to the gui class, otherwise it gets garbage collected
#     gui.image = tkimg
#     canvas = gui.canvas
#     canvas.configure(image=tkimg)
#     canvas.grid(row=0, column=1, padx=5)
#     # canvas.pack(side="bottom", fill="both", expand="yes")
# Footer
# # © 2023 GitHub, Inc.
# Footer navigation


ModuleNotFoundError: No module named 'gui'